In [57]:
from epw import epw #https://github.com/building-energy/epw

epw_trd = epw()
epw_trd.read("./weather/NOR_TD_Trondheim-Voll.012570_TMYx.2007-2021.epw")
epw_trd_df = epw_trd.dataframe
epw_trd_df.head(2)

,Year,Month,Day,Hour,Minute,Data Source and Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,...,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
0,2009,1,1,1,0,?9?9?9?9E0?9?9?9*9*9?9?9?9?9?9?9*9?9*9*9?9*9,-2.3,-2.4,99,100865,...,314,9,999999999,8,0.085,20,88,0.43,0.2,0.0
1,2009,1,1,2,0,?9?9?9?9E0?9?9?9*9*9?9?9?9?9?9?9*9?9*9*9?9*9,-2.4,-2.9,96,100894,...,360,9,999999999,7,0.085,20,88,0.44,0.2,0.0


In [58]:
import pandas as pd
from datetime import timedelta
temp_trd = pd.read_csv("./weather/1OutdoorEntrance.csv")
temp_trd = temp_trd.drop(temp_trd.columns[0], axis=1)

#Convert to daylight saving time since we eill do the simulations during summer(NB: for some reason 'Ect/GMT-2 correspond to GMT+2)
temp_trd['timestamp'] = pd.to_datetime(temp_trd['timestamp'],  utc=True).dt.tz_convert('Etc/GMT-2') 
temp_trd = temp_trd.set_index("timestamp").resample('1H').mean().interpolate().reset_index()
print(f"BEFORE:\nHEAD:\n{temp_trd.head(2)}\n\nTAIL:\n{temp_trd.tail(2)}")

temp_trd["Year"] = temp_trd["timestamp"].dt.year
temp_trd["Month"] = temp_trd["timestamp"].dt.month
temp_trd["Day"] = temp_trd["timestamp"].dt.day
temp_trd["Hour"] = temp_trd["timestamp"].dt.hour
temp_trd["Minute"] = temp_trd["timestamp"].dt.minute
#Change from timestamps from 00.00 to 24.00 the day before due to EPW format
temp_trd.loc[temp_trd["Hour"] == 0, "Month"] = (temp_trd.loc[temp_trd["timestamp"].dt.hour == 0]["timestamp"] - timedelta(1)).dt.month
temp_trd.loc[temp_trd["Hour"] == 0, "Day"] = (temp_trd.loc[temp_trd["timestamp"].dt.hour == 0]["timestamp"] - timedelta(1)).dt.day
temp_trd.loc[temp_trd["Hour"] == 0, "Hour"] = 24

temp_trd = temp_trd.rename(columns={"1OutdoorEntrance": "Dry Bulb Temperature"})
temp_trd = temp_trd.drop(columns="timestamp")

print(f"\n\nAFTER:\n{temp_trd.head(2)}")

BEFORE:
HEAD:
                  timestamp  1OutdoorEntrance
0 2022-01-07 21:00:00+02:00             -2.75
1 2022-01-07 22:00:00+02:00             -2.60

TAIL:
                     timestamp  1OutdoorEntrance
5662 2022-08-31 19:00:00+02:00           14.2625
5663 2022-08-31 20:00:00+02:00           13.1875


AFTER:
   Dry Bulb Temperature  Year  Month  Day  Hour  Minute
0                 -2.75  2022      1    7    21       0
1                 -2.60  2022      1    7    22       0


In [59]:
temp_trd["day_encoded"] = 31*24*(temp_trd['Month']-1) + 24*(temp_trd['Day']-1) + temp_trd['Hour']-1
temp_trd = temp_trd.set_index("day_encoded")
epw_trd_df["day_encoded"] = 31*24*(epw_trd_df['Month']-1) + 24*(epw_trd_df['Day']-1) + epw_trd_df['Hour']-1
epw_trd_df = epw_trd_df.set_index("day_encoded")

updated_frame = epw_trd_df.loc[(epw_trd_df.index >= temp_trd.index[0]) & (epw_trd_df.index <= temp_trd.index[-1])].copy()
updated_frame["Dry Bulb Temperature"] = temp_trd["Dry Bulb Temperature"]
updated_frame["Year"] = temp_trd["Year"].astype('int')

epw_trd_df.loc[(epw_trd_df.index >= updated_frame.index[0]) & (epw_trd_df.index <= updated_frame.index[-1])] = updated_frame
epw_trd_df.loc[epw_trd_df.index >= updated_frame.index[0] - 1].head(2) #Start of updated temperatures


,Year,Month,Day,Hour,Minute,Data Source and Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,...,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
day_encoded,,,,,,,,,,,,,,,,,,,,,
163,2009,1,7,20,0,?9?9?9?9E0?9?9?9*9*9?9?9?9?9?9?9*9?9*9*9?9*9,-4.00,-4.4,97,99734,...,907,9,999999999,6,0.084,32,88,0.54,0.1,0.0
164,2022,1,7,21,0,?9?9?9?9E0?9?9?9*9*9?9?9?9?9?9?9*9?9*9*9?9*9,-2.75,-4.9,92,99724,...,603,9,999999999,6,0.084,32,88,0.54,0.2,0.0


In [60]:
epw_trd.dataframe = epw_trd_df
epw_trd.write('./weather/NOR_TD_Trondheim_updated.epw')